In [1]:
import os
from keras.preprocessing.image import array_to_img, img_to_array, ImageDataGenerator
from keras.models import Sequential
from keras.layers import InputLayer, Conv2D, UpSampling2D
import numpy as np
from skimage.color import rgb2lab, lab2rgb
from PIL import Image

In [ ]:
testDir = ''
trainDir = ''
valDir = ''
trainSet = []
testSet = []
valSet = []

#Image.open('/Volumes/3515395439/Colorization/test2017/000000339995.jpg')
for index, image in enumerate(os.listdir(testDir)):
    if not image.startswith('._'):
        imagearr = img_to_array(Image.open(testDir + image).convert('RGB').resize((256,256)))
        testSet.append(imagearr)
    if len(testSet) == 100:
        break
for index, image in enumerate(os.listdir(trainDir)):
    if not image.startswith('._'):
        imagearr = img_to_array(Image.open(trainDir + image).convert('RGB').resize((256,256)))
        trainSet.append(imagearr)
    if len(trainSet) == 20000:
        break
for index, image in enumerate(os.listdir(valDir)):
    if not image.startswith('._'):
        imagearr = img_to_array(Image.open(valDir + image).convert('RGB').resize((256,256)))
        valSet.append(imagearr)
    if len(valSet) == 5000:
        break

trainSet = (1.0/255) * np.array(trainSet,dtype=float)
testSet = (1.0/255) * np.array(testSet,dtype=float)
valSet = (1.0/255) * np.array(valSet,dtype=float)

In [ ]:
model = Sequential()
model.add(InputLayer(input_shape=(256, 256, 1)))
model.add(Conv2D(64, (3, 3), activation='leaky_relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='leaky_relu', padding='same', strides=2))
model.add(Conv2D(128, (3, 3), activation='leaky_relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='leaky_relu', padding='same', strides=2))
model.add(Conv2D(256, (3, 3), activation='leaky_relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='leaky_relu', padding='same', strides=2))
model.add(Conv2D(512, (3, 3), activation='leaky_relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='leaky_relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='leaky_relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='leaky_relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='leaky_relu', padding='same'))
model.add(Conv2D(2, (3, 3), activation='sigmoid', padding='same'))
model.add(UpSampling2D((2, 2)))
# Finish model
model.compile(optimizer='rmsprop', loss='mse')

In [22]:
datagen = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    rotation_range=20,
    horizontal_flip=True)
# Generate training data

batch_size = 50
def image_a_b_gen(batch_size):
    for batch in datagen.flow(trainSet, batch_size=batch_size):
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield X_batch.reshape(X_batch.shape + (1,)), Y_batch
# Train model

model.fit(image_a_b_gen(batch_size), steps_per_epoch=2000, epochs=20)

Epoch 1/10


2023-07-05 00:41:01.973187: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2000/2000 [==============================] - 2492s 1s/step - loss: 0.0155
Epoch 2/10
2000/2000 [==============================] - 2483s 1s/step - loss: 0.0122
Epoch 3/10
2000/2000 [==============================] - 2803s 1s/step - loss: 0.0119
Epoch 4/10
2000/2000 [==============================] - 2405s 1s/step - loss: 0.0117
Epoch 5/10
2000/2000 [==============================] - 2408s 1s/step - loss: 0.0116
Epoch 6/10
2000/2000 [==============================] - 2407s 1s/step - loss: 0.0115
Epoch 7/10
2000/2000 [==============================] - 2408s 1s/step - loss: 0.0114
Epoch 8/10
2000/2000 [==============================] - 2420s 1s/step - loss: 0.0114
Epoch 9/10
2000/2000 [==============================] - 2409s 1s/step - loss: 0.0112
Epoch 10/10
2000/2000 [==============================] - 2408s 1s/step - loss: 0.0111


In [23]:
model.save('model20k.h5')

/Users/alisa/opt/anaconda3/lib/python3.9/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [24]:
from skimage.io import imsave
from keras.models import load_model

model = load_model('model20k.h5')

batch_size = 50
Xtest= rgb2lab(valSet)[:,:256,:256,0]
Xtest = Xtest.reshape(Xtest.shape+(1,))
Ytest = rgb2lab(valSet)[:,:256,:256,1:]
Ytest = Ytest / 128
print (model.evaluate(Xtest, Ytest, batch_size=batch_size))
# Load black and white images
color_me = []
color_me = rgb2lab(testSet)[:,:256,:256,0]
color_me = color_me.reshape(color_me.shape+(1,))
# Test model
output = model.predict(color_me)
output = output * 128
# Output colorizations
for i in range(len(output)):
    cur = np.zeros((256, 256, 3))
    cur[:,:,0] = color_me[i][:,:,0]
    cur[:,:,1:] = output[i]
    imsave("result/img_"+str(i)+".png", lab2rgb(cur))


2023-07-05 07:32:11.897736: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


20/20 [==============================] - 6s 251ms/step - loss: 0.0129
0.012918348424136639


2023-07-05 07:32:19.919042: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


4/4 [==============================] - 2s 160ms/step


Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0